# StorIA-LITE API Usage Guide

This Jupyter Notebook provides a hands-on guide to interacting with the StorIA-LITE backend API. We will walk through the common workflows, from registering and authenticating a user to performing CRUD (Create, Read, Update, Delete) operations on inventory items.

**Prerequisites:**
1. The StorIA-LITE backend server must be running (see the main `README.md` for instructions).
2. You need to have Python installed with the `requests` library (`pip install requests`).

## Step 1: Setup and Configuration

First, let's import the necessary libraries and define the base URL for our API. If you are running the server on a different port, make sure to update the `BASE_URL` variable.

In [ ]:
import requests
import json
from datetime import datetime, timedelta
import random
import string

# The base URL of the running API
BASE_URL = "https://localhost:7123/api"

# We will store the JWT token here after login
auth_token = None

# Helper function to print JSON responses nicely
def print_json(data):
    print(json.dumps(data, indent=2))

# Helper to generate random strings for user/item creation to ensure uniqueness
def random_string(length=8):
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=length))

## Step 2: User Registration

Before we can do anything that requires authentication, we need to register a new user. We'll create a user with the `Almoxarife` role, which has permissions to manage items.

In [ ]:
registration_url = f"{BASE_URL}/Auth/register"

# Generate a unique email for the new user
unique_email = f"testuser_{random_string()}@example.com"

new_user = {
    "fullName": "Test User",
    "email": unique_email,
    "password": "Password123!",
    "role": "Almoxarife" # Other roles: RH, Gestor
}

print(f"Attempting to register user: {unique_email}")
response = requests.post(registration_url, json=new_user, verify=False) # verify=False for development with self-signed certs

if response.status_code == 201:
    print("✅ User registered successfully!")
    print_json(response.json())
else:
    print(f"❌ User registration failed with status code: {response.status_code}")
    print(response.text)

## Step 3: User Login

Now that we have a user, let's log in to get a JSON Web Token (JWT). This token is required to access protected endpoints.

In [ ]:
login_url = f"{BASE_URL}/Auth/login"

login_credentials = {
    "email": new_user['email'],
    "password": new_user['password']
}

print(f"Attempting to log in as: {login_credentials['email']}")
response = requests.post(login_url, json=login_credentials, verify=False)

if response.status_code == 200:
    print("✅ Login successful!")
    auth_data = response.json()
    auth_token = auth_data.get('token')
    print("--- Authentication Token ---")
    print(auth_token)
else:
    print(f"❌ Login failed with status code: {response.status_code}")
    print(response.text)

## Step 4: Creating a New Item

With our `auth_token`, we can now access protected endpoints. Let's create a new inventory item. We need to include the token in the `Authorization` header of our request.

In [ ]:
items_url = f"{BASE_URL}/Items"

headers = {
    "Authorization": f"Bearer {auth_token}"
}

new_item = {
    "name": f"Safety Helmet - {random_string(4)}",
    "sku": f"SKU-{random_string(8)}",
    "description": "A durable and high-quality safety helmet.",
    "category": "PPE",
    "initialQuantity": 100,
    "expiryDate": (datetime.now() + timedelta(days=365)).isoformat()
}

print("Attempting to create a new item...")
response = requests.post(items_url, json=new_item, headers=headers, verify=False)

if response.status_code == 201:
    print("✅ Item created successfully!")
    created_item = response.json()
    item_id = created_item.get('id') # Save the ID for later steps
    print_json(created_item)
else:
    print(f"❌ Item creation failed with status code: {response.status_code}")
    print(response.text)

## Step 5: Retrieving Items

Let's fetch a list of all items to see the one we just created. This endpoint supports pagination.

In [ ]:
print("Attempting to retrieve all items...")
# Let's get the first page with 5 items per page
query_params = {"pageNumber": 1, "pageSize": 5}

response = requests.get(items_url, headers=headers, params=query_params, verify=False)

if response.status_code == 200:
    print("✅ Items retrieved successfully!")
    print_json(response.json())
else:
    print(f"❌ Failed to retrieve items. Status code: {response.status_code}")
    print(response.text)

## Step 6: Updating an Item

Now, let's update the description and stock quantity of the item we created earlier.

In [ ]:
update_item_url = f"{items_url}/{item_id}"

update_payload = {
    "description": "An updated, high-visibility safety helmet.",
    "stockQuantity": 150
}

print(f"Attempting to update item with ID: {item_id}")
response = requests.put(update_item_url, json=update_payload, headers=headers, verify=False)

if response.status_code == 204:
    print("✅ Item updated successfully!")
    # Let's fetch the item again to verify the changes
    verify_response = requests.get(update_item_url, headers=headers, verify=False)
    if verify_response.status_code == 200:
        print("--- Verified Item Data ---")
        print_json(verify_response.json())
else:
    print(f"❌ Failed to update item. Status code: {response.status_code}")
    print(response.text)

## Step 7: Deleting an Item

Finally, let's clean up by deleting the item we created. Note that for this to work, the user's role must have delete permissions (e.g., `RH`). If our user `Almoxarife` doesn't have this permission, this request will fail with a `403 Forbidden` status.

In [ ]:
# To test deletion, you might need to register/login as a user with the 'RH' role.
# For this example, we'll assume the current user might not have permission.

delete_item_url = f"{items_url}/{item_id}"

print(f"Attempting to delete item with ID: {item_id}")
response = requests.delete(delete_item_url, headers=headers, verify=False)

if response.status_code == 204:
    print("✅ Item deleted successfully!")
elif response.status_code == 403:
    print("⚠️ Could not delete item. Status code 403: Forbidden. The current user's role likely lacks permission.")
else:
    print(f"❌ Failed to delete item. Status code: {response.status_code}")
    print(response.text)